# Exam Learning Path - Creation Tool

## Imports

In [ ]:
// Hard Coding Versions For Consistency
#r "nuget: FSharp.Data, 5.0.2"

In [ ]:
open System.Linq
open FSharp.Data
open System.Net.Http
open System.Text.RegularExpressions

## Domain

In [ ]:
type DisplayName = | DisplayName of string
    with
      override this.ToString() = this |> function DisplayName str -> str
let DisplayName str =
    if Regex.IsMatch(str, "^[a-z0-9]{2,3}-[a-z0-9]{3}$", RegexOptions.IgnoreCase) then
        DisplayName (str.ToUpperInvariant())
    else
        failwith $"A display name should follow a pattern like \"AB-123\", but was {str}"

type Exam = {
    Title: string
    DisplayName: DisplayName
    Link: string
    State: ExamState
    LearningPaths: LearningPath list
}

and ExamState =
    | Current
    | Retired
    | RetiredWithReplacement of Exam
    | RetiredOn of DateOnly
    | RetiredOnWithReplacments of DateOnly * Exam list

and LearningPath =
    { Title: string
      Link: string
      Duration: TimeSpan
      Modules: Module list }
      static member Default =
          { Title = "Missing"
            Link = "Missing"
            Duration = TimeSpan.FromSeconds(0)
            Modules = [] }

and Module =
    { Title: string
      Link: string
      Duration: TimeSpan }
      static member Default =
          { Title = "Missing"
            Link = "Missing"
            Duration = TimeSpan.FromSeconds(0) }

## Helpers

In [ ]:
let toAbsoluteURL (pageURL: string) (urlToUpdate: string) : string =
    let pageURI = Uri(pageURL)
    match Uri.TryCreate(pageURI, urlToUpdate) with
    | true, uri -> uri.ToString()
    | false, _  -> failwith $"urlToUpdate value is in an unexpected format. urlToUpdate = {urlToUpdate}"

let makeCacheable0 func =
    let mutable cell = None
    fun () -> match cell with
                | Some value -> value
                | None ->
                    let value = func ()
                    cell <- Some value
                    value

let makeCacheable1 func =
    let mutable map = Map.empty
    fun arg0 -> match map.TryGetValue(arg0) with
                | true, value -> value
                | false, _ ->
                    let value = func arg0
                    map <- map |> Map.add arg0 value
                    value

type DisplayStopWatch() =
    let sw = System.Diagnostics.Stopwatch.StartNew()
    let mutable current = None

    let completed () : unit =
        match current with
        | None -> ()
        | Some name -> $"Completed {name} at {sw.Elapsed.ToString()}".Display() |> ignore

    member this.Start(name: string) : unit =
        completed ()
        current <- Some name
        name.Display() |> ignore

    interface IDisposable with
        member this.Dispose() : unit =
            completed ()
            sw.Stop()

### JSON

In [ ]:
module WebAPI =
    let [<Literal>] private ROOT_SEARCH_URL = "https://docs.microsoft.com/api/contentbrowser/search/certifications?locale=en-us&$orderBy=title&$top=30"
    let [<Literal>] private ROOT_EXAM_URL = "https://docs.microsoft.com/api/lists/studyguide/exam/exam.mb-340?locale=en-us"
    let [<Literal>] private ROOT_LEARNINGPATH_URL = "https://docs.microsoft.com/api/hierarchy/paths/learn.wwl.examine-core-capabilities-of-microsoft-dynamics-365-crm?locale=en-us"
    let [<Literal>] private ROOT_MODULE_URL = "https://docs.microsoft.com/api/hierarchy/paths/learn-dynamics.get-started-dynamics-365-commerce?locale=en-us"

    type SearchWebAPI = JsonProvider<ROOT_SEARCH_URL>
    type ExamWebAPI = JsonProvider<ROOT_EXAM_URL>
    type LearningPathWebAPI = JsonProvider<ROOT_LEARNINGPATH_URL>
    type ModuleWebAPI = JsonProvider<ROOT_MODULE_URL>


    let getSearchData () : SearchWebAPI.Result list =
        let staging = System.Collections.Generic.List<SearchWebAPI.Result>()
        
        let mutable url = ROOT_SEARCH_URL
        while not (String.IsNullOrWhiteSpace(url)) do
            url <- toAbsoluteURL ROOT_SEARCH_URL url

            let res = SearchWebAPI.AsyncLoad(url) |> Async.RunSynchronously
            staging.AddRange(res.Results)
            url <- res.NextLink

        staging.DistinctBy(fun x -> x.Uid)
        |> List.ofSeq

    let getExamData (examUid: string) : ExamWebAPI.Root =
        let url = ROOT_EXAM_URL.Replace("exam.mb-340", examUid)
        ExamWebAPI.AsyncLoad(url) |> Async.RunSynchronously

    let getLearningPathData (learningPathId: string) : LearningPathWebAPI.Root =
        let url = ROOT_LEARNINGPATH_URL.Replace("learn.wwl.examine-core-capabilities-of-microsoft-dynamics-365-crm", learningPathId)
        LearningPathWebAPI.AsyncLoad(url) |> Async.RunSynchronously

    let getModuleData (moduleId: string) : ModuleWebAPI.Root =
        let url = ROOT_LEARNINGPATH_URL.Replace("learn-dynamics.get-started-dynamics-365-commerce", moduleId)
        ModuleWebAPI.AsyncLoad(url) |> Async.RunSynchronously


### HTML

In [ ]:
module WebPage =
    open System.Text.RegularExpressions

    let [<Literal>] private WEBPAGE_URL_TEMPLATE = "https://docs.microsoft.com/en-us/certifications/exams/{DisplayName}"

    type RetirementStatus =
        | Current
        | Retired
        | RetiredWithReplacement of examDisplayName:DisplayName
        | RetiredOn of DateOnly
        | RetiredOnWithReplacements of DateOnly * examDisplayNames:DisplayName list

    let getRetirementStatusFromHTML (html: string) : RetirementStatus =
        // Ignore the word "Error"
        // We're using the Result type for its monadic characteristics

        let retired (html: string) =
            if html.Contains("This exam has been retired. For currently available options, please refer to", StringComparison.OrdinalIgnoreCase) 
               || html.Contains("THIS EXAM IS RETIRED. For currently available options, please refer to", StringComparison.OrdinalIgnoreCase)
               || html.Contains("This exam is retired. Please refer to") then
                Error Retired
            else
                Ok html

        let replacement html =
            let genMatch regexStr =
                fun html ->
                    match Regex.Match(html, regexStr).Groups[1].Value with
                    | "" -> Ok html
                    | str -> Error (RetiredWithReplacement (DisplayName str))

            Ok html
            |> Result.bind (genMatch "THIS EXAM IS RETIRED\. A replacement exam, (.{2:3}-.*?),")
            |> Result.bind (genMatch "This exam is retired\. A replacement exam, <a.*?>Exam (.*?):")
            |> Result.bind (genMatch "THIS EXAM IS RETIRED\. A replacement exam, (.*?), is available")

        let date html =
            let genMatch regexStr =
                fun html ->
                    match Regex.Match(html, regexStr).Groups[1].Value with
                    | ""  -> Ok html
                    | str -> Error (RetiredOn (DateOnly.Parse(str)))

            Ok html
            |> Result.bind (genMatch "This exam retired on ([a-zA-Z]+ [0-9]{1,2}, [0-9]{4})")
            |> Result.bind (genMatch "THIS EXAM WAS RETIRED (.*?)\. Note:")

        let dateAndReplacement html =
            let genMatch regexStr =
                fun html -> 
                    let groups = Regex.Match(html, regexStr).Groups
                    match groups[1].Value, groups[2].Value with
                    | "", ""  -> Ok html
                    | dateStr, examStr -> Error (RetiredOnWithReplacements (DateOnly.Parse(dateStr), [DisplayName examStr]))

            let genMatches regexStr =
                fun html -> 
                    let groups = Regex.Match(html, regexStr).Groups
                    match groups[1].Value, groups[2].Value, groups[3].Value with
                    | "", "", ""  -> Ok html
                    | dateStr, examStr1, examStr2 -> Error (RetiredOnWithReplacements (DateOnly.Parse(dateStr), [DisplayName examStr1; DisplayName examStr2]))

            Ok html
            |> Result.bind (genMatch "THIS EXAM RETIRED ON (.*?)\. A replacement exam, (.*?),")
            |> Result.bind (genMatch "This exam retired on (.*?)\. A replacement exam, (.*?), is available\.")
            |> Result.bind (genMatch "THIS EXAM RETIRED ON (.*?)\. A new version of this exam, (.*?), is available")
            |> Result.bind (genMatch "Starting on (.*?, [0-9]{4}), you only need to pass Exam (.*?) to earn")
            |> Result.bind (genMatches "THIS EXAM RETIRED ON (.*?)\. Two replacement exams, (.*?) and (.*?), are available")


        Ok html
        |> Result.bind retired
        |> Result.bind replacement
        |> Result.bind date
        |> Result.bind dateAndReplacement
        |> function
           | Error value -> value
           | Ok _        -> Current

    let getExamHtml (examDisplayName: DisplayName) : string =
        let url = WEBPAGE_URL_TEMPLATE.Replace("{DisplayName}", examDisplayName.ToString())
        use http = new HttpClient()
        http.GetStringAsync(url).Result

    let getExamRetirementStatus (examDisplayName: DisplayName) : RetirementStatus =
        getExamHtml examDisplayName
        |> getRetirementStatusFromHTML
                            

## Crawl

Entry Point -> Domain Objects

In [ ]:
module CachedWebAPI =
    let getSearchData       = makeCacheable0 (WebAPI.getSearchData)
    let getExamData         = makeCacheable1 (WebAPI.getExamData)
    let getLearningPathData = makeCacheable1 (WebAPI.getLearningPathData)
    let getModuleData       = makeCacheable1 (WebAPI.getModuleData)

module CachedWebPage =
    let getExamHtml             = makeCacheable1 (WebPage.getExamHtml)
    let getExamRetirementStatus = makeCacheable1 (WebPage.getExamRetirementStatus)

// Alias the cached versions so we can iterate
// on the Crawl module without hitting the network
module WebAPI = CachedWebAPI
module WebPage = CachedWebPage

In [ ]:
module Crawl =
    open System.Net

    let [<Literal>] ROOT_URL = "https://docs.microsoft.com/en-us/"
   
    type TempExam = {
        Title: string
        DisplayName: DisplayName
        Link: string
        State: WebPage.RetirementStatus
        LearningPaths: LearningPath list
    }

    module Convert =
        open WebPage
        open System.Collections.Generic

        let toExam (tempExam: TempExam) (examState: ExamState) : Exam =
            { Exam.Title = tempExam.Title
              DisplayName = tempExam.DisplayName
              Link = tempExam.Link
              State = examState
              LearningPaths = tempExam.LearningPaths }

        let toExams (tempExams: TempExam list) : Exam list =
            let exams = Dictionary<DisplayName, Exam>()
            let queue = Queue(tempExams)

            while queue.Count > 0 do
                let tempExam = queue.Dequeue()

                match tempExam.State with
                | Current -> Some (ExamState.Current)
                | Retired -> Some (ExamState.Retired)
                | RetiredOn date -> Some (ExamState.RetiredOn date)
            
                | RetiredWithReplacement displayName ->
                    if exams.ContainsKey(displayName) then
                        Some (ExamState.RetiredWithReplacement (exams[displayName]))
                    else
                        None
            
                | RetiredOnWithReplacements (date, displayNames) ->
                    if displayNames |> List.forall exams.ContainsKey then
                        let exams = displayNames |> List.map (fun displayName -> exams[displayName])
                        Some (ExamState.RetiredOnWithReplacments (date, exams))
                    else
                        None

                |> function
                   | None           -> queue.Enqueue(tempExam)
                   | Some examState -> exams.Add(tempExam.DisplayName, (toExam tempExam examState))

            exams.Values
            |> Seq.sortBy (fun exam -> exam.DisplayName.ToString())
            |> List.ofSeq

    let private webAPI_getLearningPathData id =
        try
            WebAPI.getLearningPathData id
            |> Some
        with
        | :? WebException -> None

    let crawl () : Exam list =
        use dsw = new DisplayStopWatch()
        let start = dsw.Start

        start "searchData"
        let searchData = WebAPI.getSearchData ()

        start "searchCertificationData"
        let searchCertificationData = searchData.Where(fun x -> x.ResourceType = "certification")
                                                .Select(fun x -> (x.Uid, x))
                                                |> Map.ofSeq

        start "searchExamData"
        let searchExamData = searchData.Where(fun x -> x.ResourceType = "examination")
                                       .Select(fun x -> (x.Uid, x))
                                       |> Map.ofSeq

        start "examData"
        let examData = searchExamData.Keys
                                     .Select(fun uid -> (uid, WebAPI.getExamData uid))
                                     |> Map.ofSeq

        start "examLearningPathData"
        let examLearningPathData = examData.Values
                                           .SelectMany(fun x -> x.Items |> Seq.ofArray)
                                           .Select(fun x -> (x.Id, x))
                                           |> Map.ofSeq

        start "learningPathData"
        let learningPathData = examLearningPathData.Keys
                                                   .Select(fun id -> (id, webAPI_getLearningPathData id))
                                                   |> Map.ofSeq

        start "learningPathModuleData"
        let learningPathModuleData = learningPathData.Values
                                                     .Where(Option.isSome)
                                                     .Select(Option.get)
                                                     .SelectMany(fun x -> x.Modules |> Seq.ofArray)
                                                     .Select(fun x -> (x.Uid, x))
                                                     |> Map.ofSeq

        let toAbsoluteURL = toAbsoluteURL ROOT_URL

        start "modules"
        let modules = learningPathModuleData
                      |> Map.map (fun id x -> { Title = x.Title
                                                Link = toAbsoluteURL x.Url
                                                Duration = TimeSpan.FromMinutes(x.DurationInMinutes) })
        start "learningPaths"
        let learningPaths = learningPathData
                            |> Map.map (fun uid x -> match x with
                                                     | None -> LearningPath.Default
                                                     | Some x -> { Title = x.Title
                                                                   Link = toAbsoluteURL x.Url
                                                                   Duration = TimeSpan.FromMinutes(x.DurationInMinutes)
                                                                   Modules = x.Modules
                                                                              .Select(fun x -> modules[x.Uid])
                                                                              |> List.ofSeq })

        start "tempExams"
        let tempExams = searchExamData
                        |> Map.map (fun uid x -> { Title = x.Title
                                                   DisplayName = DisplayName x.ExamDisplayName
                                                   Link = toAbsoluteURL x.Url
                                                   State = WebPage.getExamRetirementStatus (DisplayName x.ExamDisplayName)
                                                   LearningPaths = (examData[uid]).Items
                                                                                  .Select(fun x -> learningPaths[x.Id])
                                                                                  |> List.ofSeq })

        start "Convert.toExams"
        Convert.toExams (tempExams |> Map.values |> List.ofSeq)


## Grouping

Group modules into weeks at some sustainable level

In [ ]:
module Grouping =

    let min = TimeSpan.FromMinutes(60)
    let max = TimeSpan.FromMinutes(90)

    let groupModulesIntoWeeks (exam: Exam) =
        let mutable dur = TimeSpan.FromSeconds(0)
        let mutable week = 1

        let flat = exam.LearningPaths
                       .SelectMany(fun lp -> lp.Modules
                                               .Select(fun m -> (lp, m)))

        seq {
            for (lp, m) in flat do
                if ((m.Duration > max || dur.Add(m.Duration) > max) && dur <> TimeSpan.FromSeconds(0)) then
                    dur <- TimeSpan.FromSeconds(0)
                    week <- week + 1

                dur <- dur.Add(m.Duration)

                yield (lp, m, week)
        }
        |> List.ofSeq

## File System

In [ ]:
module FileSystem =
    
    let readmePath = "README.md"

    let readmeTemplatePath = "README.md.template"

    let examsPath = "Exams"

    let examPath (exam: Exam) : string =
        $"Exams/{exam.DisplayName}.md"

    let init () : unit =
        if File.Exists(readmePath) then
            File.Delete(readmePath)

        if Directory.Exists(examsPath) then
            Directory.Delete(examsPath, true)

        System.Threading.Thread.Sleep(250)

        Directory.CreateDirectory(examsPath) |> ignore


## Markdown

In [ ]:
module Markdown =

    let h1 str = "# " + str
    let boldStr str = "**" + str + "**"
    let boldInt i   = "**" + (string i) + "**"
    let url text url = "[" + text + "](" + url + ")"
    let pad str = " " + str + " "
    let emptyCell = " "

    let examFullName (exam: Exam) : string =
        $"{exam.DisplayName}: {exam.Title}"

    let exam (exam: Exam) : string =
        let sb = StringBuilder()
        
        sb.AppendLine(h1 (examFullName exam))
          .AppendLine()
          .AppendLine($"**Exam/Learning Path:** {exam.Link}")
          .AppendLine()
          |> ignore

        sb.AppendLine("| **Learning Path** | **Module** | **Week** |")
          .AppendLine("|-|-|-|")
          |> ignore

        let mutable previousLeaarningPathLink = null
        for (lp, m, w) in Grouping.groupModulesIntoWeeks exam do
            let lpCell = if lp.Link = previousLeaarningPathLink then
                            emptyCell
                         else
                            previousLeaarningPathLink <- lp.Link
                            boldStr (url lp.Title lp.Link)

            let mCell = pad (url m.Title m.Link)

            let wCell = pad (boldInt w)

            sb.AppendLine($"|{lpCell}|{mCell}|{wCell}")
              |> ignore

        sb.ToString()

    let examsLinks (exams: Exam list) : string =
        let sb = StringBuilder()
        
        for exam in exams do
            sb.Append(url (examFullName exam) (FileSystem.examPath exam))
              .AppendLine("<br/>")
              |> ignore
        
        sb.ToString()

    let readme (readmeTemplate: string, exams: Exam list) : string =
        let currentExams, retiredExams = exams |> List.partition (fun e -> e.State = ExamState.Current)
        readmeTemplate.Replace("{{{Current-Exam-Links}}}", examsLinks currentExams)
                      .Replace("{{{Retired-Exam-Links}}}", examsLinks retiredExams)

## Do

In [ ]:
let exams = Crawl.crawl()

In [ ]:
FileSystem.initExamsDirectory ()

for exam in exams do
    let path = FileSystem.examPath exam
    let markdown = Markdown.exam exam
    File.WriteAllText(path, markdown)

let readmeTemplate = File.ReadAllText(FileSystem.readmeTemplatePath)
let readme = Markdown.readme(readmeTemplate, exams)
File.WriteAllText(FileSystem.readmePath, readme)